# Connectivity Cost Estimation

> For help, see documentation on [configuring and running these models](../README.md). 

<a id="top"></a>


In [ ]:
from dotenv import load_dotenv
load_dotenv()
import json
import os
from ipywidgets import FileUpload, Output, Label

from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload

import warnings
warnings.filterwarnings('ignore')


show_map = os.environ.get("FF_STATIC_MAP_NOTEBOOK", True)

output = Output()

upload = GigaFileUpload(accept='.json', multiple=True, description="Import Config")
inputs = CostEstimationParameterInput(local_data_workspace='workspace', show_map=show_map)

def on_upload_change(change):
    try:
        imported = json.loads(upload.value[0].content.tobytes())
    except IndexError:
        imported = {}
    with output:
        output.clear_output()
        inputs.update(imported)
    display(imported)
    display(upload, inputs.parameter_input(), output)

upload.observe(on_upload_change, names='value')
display(upload, inputs.parameter_input(), output)

## II. Run the Model

In [ ]:
from IPython.display import display, clear_output
from ipywidgets import Button, Output, HBox
from tqdm import tqdm
from IPython.display import clear_output
import sys
import time
import logging
import geopandas as gpd

import json
from io import StringIO
from io import BytesIO


from giga.models.scenarios.scenario_dispatcher import create_scenario
from giga.data.space.model_data_space import ModelDataSpace
from giga.schemas.output import OutputSpace
from giga.utils.logging import LOGGER
from giga.viz.notebooks.tables import display_summary_table, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table
from giga.viz.notebooks.components.widgets.giga_buttons import make_run_button, make_run_again_button
from giga.viz.notebooks.components.widgets.giga_export import make_export_button_row
from giga.viz.notebooks.components.dashboard.result_dashboard import ResultDashboard

result_output = Output()

output_space = None
data_space = None
config = None
button = None


def on_button_clicked(b):
    with result_output:
        clear_output(wait=True)
        
        global config
        global data_space
        global output_space
        # freeze and disable confgiuration + running while model in progress
        b.disabled = True
        inputs.freeze()
        # pull verbose flag
        verbose = inputs.dashboard_parameters()["verbose"]
        LOGGER.propagate = verbose
        # setup scenario
        config = inputs.scenario_parameters()
        data_space = ModelDataSpace(inputs.data_parameters())
        output_space = OutputSpace()
        scenario = create_scenario(config, data_space, output_space)
        # run the models
        output_space = scenario.run(progress_bar=verbose)
        table = output_to_table(output_space)
        table_geo = data_space.school_outputs_to_frame(table)
        # show summary tables
        display_summary_table(output_space, data_space)
        # start the dashboard
        dashboard = ResultDashboard(table_geo)
        dashboard.display()
        # make export + rerun buttons
        export_buttons = make_export_button_row(table, inputs)
        run_again_button = make_run_again_button(inputs, result_output, b)
        display(export_buttons)
        display(run_again_button)

button = make_run_button(on_button_clicked)
button.on_click(on_button_clicked)
display(button, result_output)

## III. Results

In [ ]:
import geopandas as gpd
from ipywidgets import HTML

from giga.viz.notebooks.tables import display_summary_table, output_to_capex_details, plot_cost_breakdown
from giga.viz.notebooks.helpers import output_to_table, button_cb
from giga.viz.notebooks.maps import show_cost_map, show_electricity_map
from giga.viz.notebooks.fiber import plot_fiber_connections, default_rwanda_map, default_map


def action():
    with result_output:
        if inputs.data_parameters().school_data_conf.country_id == 'brazil':
            display(HTML("<hr><b><font color='#5b8ff0'>Unable to display maps for Brazil</b><br><br>"))
            return

        center = inputs.defaults[inputs.data_parameters().school_data_conf.country_id].data.country_center_tuple

        display(HTML("<hr><b><font color='#5b8ff0'>Electricity Map</b><br><br>"))
        display(show_electricity_map(data_space, location=center))

        display(HTML("<hr><b><font color='#5b8ff0'>Cost Map</b><br><br>"))
        display(show_cost_map(data_space, output_space, location=center))

        if output_space.fiber_costs is not None:
            m = default_map(location=center)
            display(HTML("<hr><b><font color='#5b8ff0'>Fiber Map</b><br><br>"))
            display(plot_fiber_connections(data_space.fiber_coordinates, data_space.school_coordinates, output_space.fiber_costs.technology_results.distances, m=m))
        

button, click = button_cb("Show Maps", action)
button.on_click(click)

In [ ]:
from IPython.display import display, HTML
from giga.viz.notebooks.helpers import output_to_table, button_cb


def action():
    with result_output:
        table = output_to_table(output_space)
        display(HTML('<h2>Cost Table</h2>'), table)


button, click = button_cb("Show Full Table", action)
button.on_click(click)

See [drilldown notebook](results-drilldown.ipynb)